<a href="https://colab.research.google.com/github/JKoscielecki/CoNaLa-Challenge/blob/main/My_CoNaLa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from google.colab import drive

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
!pip install transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 67.9 MB/s eta 0:00:00


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 20.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
import transformers
import datasets
from datasets import load_dataset, load_metric
import logging
from transformers import BertTokenizer, GPT2Tokenizer, GPT2TokenizerFast, EncoderDecoderModel, Trainer, TrainingArguments, BertTokenizerFast
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

logging.basicConfig(level=logging.INFO)

import types
import argparse
import logging
from functools import partial
import json

import torch
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import (
    BertGenerationConfig,
    BertGenerationEncoder,
    BertTokenizer,
    EncoderDecoderModel,
    EncoderDecoderConfig,
    GPT2LMHeadModel,
    GPT2TokenizerFast,
    Trainer,
    TrainingArguments,
    get_cosine_schedule_with_warmup,
)

import random
import numpy as np
#from tensorboardX import SummaryWriter


In [ ]:
drive.mount('/content/gdrive')
df = pd.read_csv('/content/gdrive/My Drive/CoNaLa Research/CoNaLa-Large/train.csv')

Mounted at /content/gdrive


In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset('/content/gdrive/My Drive/CoNaLa Research/CoNaLa-Large')

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/CoNaLa-Large-c31c8616c06f8dc5/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
train_dataset= raw_dataset["train"]
val_dataset = raw_dataset["validation"]
test_dataset = raw_dataset["test"]

In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.3 MB/s eta 0:00:00


# **Model and Tokenizer**

In [ ]:
from transformers import AutoConfig, AutoTokenizer,AutoModelForSeq2SeqLM
config = AutoConfig.from_pretrained("patrickvonplaten/xprophetnet-large-uncased-standalone")
tokenizer = AutoTokenizer.from_pretrained("patrickvonplaten/xprophetnet-large-uncased-standalone")
model = AutoModelForSeq2SeqLM.from_config(config)


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--patrickvonplaten--xprophetnet-large-uncased-standalone/snapshots/0d7d60dded4dc2bfab2a94e93ce37f8c84ca7028/config.json
Model config XLMProphetNetConfig {
  "_name_or_path": "patrickvonplaten/xprophetnet-large-uncased-standalone",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_cross_attention": false,
  "architectures": [
    "XLMProphetNetDecoder"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 2,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_max_position_embeddings": 514,
  "decoder_start_token_id": 2,
  "disable_ngram_loss": false,
  "dropout": 0.1,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_max_position_embeddings": 513,
  "eos_token_id": 2,
  "eps": 0.0,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_decoder": true,
  "is_encoder_decoder": false,


TypeError: ignored

In [ ]:
#@title
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoModelForSeq2SeqLM
tokenizer = AutoTokenizer.from_pretrained("patrickvonplaten/xprophetnet-large-uncased-standalone")
model = AutoModelForSeq2SeqLM.from_pretrained("patrickvonplaten/xprophetnet-large-uncased-standalone")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

Could not locate the tokenizer configuration file, will try to use the model config instead.


Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--patrickvonplaten--xprophetnet-large-uncased-standalone/snapshots/0d7d60dded4dc2bfab2a94e93ce37f8c84ca7028/config.json
Model config XLMProphetNetConfig {
  "_name_or_path": "patrickvonplaten/xprophetnet-large-uncased-standalone",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_cross_attention": false,
  "architectures": [
    "XLMProphetNetDecoder"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 2,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_max_position_embeddings": 514,
  "decoder_start_token_id": 2,
  "disable_ngram_loss": false,
  "dropout": 0.1,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_max_position_embeddings": 513,
  "eos_token_id": 2,
  "eps": 0.0,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_decoder": true,
  "is_encoder_decoder": false,


TypeError: ignored

# **Prepare the data**

In [ ]:
encoder_length = 32
decoder_length = 32
batch_size = 64


# map data correctly
def map_to_encoder_decoder_inputs(batch):    
    inputs = tokenizer(batch["intent"], padding="max_length", truncation=True, max_length=encoder_length)
    outputs = tokenizer(batch["snippet"], padding="max_length", truncation=True, max_length=decoder_length)
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()
    batch["decoder_attention_mask"] = outputs.attention_mask
    
    """
    # complicated list comprehension here because pad_token_id alone is not good enough to know whether label should be excluded or not
    batch["labels"] = [
        [-100 if mask == 0 else token for mask, token in mask_and_tokens] for mask_and_tokens in [zip(masks, labels) for masks, labels in zip(batch["decoder_attention_mask"], batch["labels"])]
    ]
    """
    assert all([len(x) == encoder_length for x in inputs.input_ids])
    assert all([len(x) == decoder_length for x in outputs.input_ids])
    
    return batch

In [ ]:
# make train dataset ready
train_data = train_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['intent', 'snippet'],
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)



  0%|          | 0/24687 [00:00<?, ?ba/s]

In [ ]:
# same for validation dataset
val_data = val_dataset.map(
    map_to_encoder_decoder_inputs, batched=True, batch_size=1, remove_columns=['intent', 'snippet'],
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


  0%|          | 0/1237 [00:00<?, ?ba/s]

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                       max_length=512,padding=True, ####new
                                       model = model)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device

device(type='cuda')

In [ ]:

username="JKoscielecki"
repository="MarianCG-NL-to-Code"
%cd {repository}
%cd Utils

/content/MarianCG-NL-to-Code
/content/MarianCG-NL-to-Code/Utils


In [ ]:
!git clone https://{git_token}@github.com/{username}/{repository}

Cloning into 'MarianCG-NL-to-Code'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (132/132), done.
remote: Total 156 (delta 64), reused 85 (delta 23), pack-reused 0
Receiving objects: 100% (156/156), 6.05 MiB | 21.08 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [ ]:
!pip install sacrebleu==1.5.1
!pip install tree-sitter
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 6.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.2/126.2 KB 13.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tree-sitter: filename=tree_sitter-0.20.1-cp38-cp38-linux_x86_64.whl size=333882 sha256=5c885122462917e50ce01ccf852893bde0ca98f197888c74363cf8c391f8efaa
  Stored in directory: /root/.cache/pip/wheels/fc/9f/3a/3a3803563accdae774c03d8d076d0f80ddcf0698bad61f010a
Successfully built tree-sitter
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 9.4 MB/s eta 0:00:00


In [ ]:
!pip install rouge-score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=905c91e463b1b04d6a94c4636a4e716559633e2da366bd40febd540dcdc1506a
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge-score


In [ ]:
import sacrebleu

In [ ]:
from google.colab import files
import evaluator
evaluator = evaluator.CodeGenerationEvaluator(tokenizer, device,smooth_bleu=True)

In [ ]:
# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    output_dir="./DistilBert-Training",
    evaluation_strategy="epoch",
    save_strategy = "epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    num_train_epochs=5,
    do_train=True,
    do_eval=True,
    fp16=True,
    overwrite_output_dir=True,
    learning_rate = 1e-5,
    weight_decay=0.01, 
    warmup_ratio = 0.05,
    seed = 1995,
    save_total_limit = 2,
    load_best_model_at_end = True,
)
#    

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=evaluator,
    data_collator = data_collator,
    train_dataset=train_data,
    eval_dataset =val_data,
       
)
#    save_total_limit=3,
# 

Using cuda_amp half precision backend


In [ ]:
# DistilBert Training
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertModel.forward` and have been ignored: decoder_input_ids, labels, decoder_attention_mask. If decoder_input_ids, labels, decoder_attention_mask are not expected by `DistilBertModel.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 24687
  Num Epochs = 5
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 61720
  Number of trainable parameters = 66362880
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:

ValueError: ignored

In [ ]:
# Instantiate the model
model = MarianModel.from_pretrained("Helsinki-NLP/opus-mt-en-ro")

# Prepare the model for training
model.prepare_seq2seq_decoder()

# Define a loss function
loss_fn = CrossEntropyLoss()

# Define an optimizer
optimizer = Adam(model.parameters(), lr=1e-5, weight_decay=0.01)

# Train the model
for epoch in range(num_train_epochs):
    # Training loop
    model.train()
    for i, (input_ids, labels) in enumerate(train_dataloader):
        # Move input and label tensors to the GPU
        input_ids = input_ids.to(device)
        labels = labels.to(device)

        # Forward pass
        logits = model(input_ids)

        # Compute the loss
        loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Evaluation loop
    model.eval()
    for i, (input_ids, labels) in enumerate(val_dataloader):
        # Move input and label tensors to the GPU
        input_ids = input_ids.to(device)
        labels = labels.to(device)

        # Forward pass
        logits = model(input_ids)

        # Compute the loss
        loss = loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
